# Assignment 2 - Part C: Trying alternative classifiers

This is a skeleton for trying alternative classifiers on the basketball dataset.

In [199]:
import csv

We can define, as done in Practicum 6, a data loading in a way to obtain the attributes set and class labels for each the training and the test sets.

In [200]:
ATTRS = ["LOCATION", "W", "FINAL_MARGIN", "SHOT_NUMBER", "PERIOD", "GAME_CLOCK", "SHOT_CLOCK", "DRIBBLES", "TOUCH_TIME",
         "SHOT_DIST", "PTS_TYPE", "CLOSE_DEF_DIST", "SHOT_RESULT"]
ATTRS_WO_CLASS = 12

OUTPUT_FILE = "data/classifier_basketball.pred.csv"

def load_data(filename):
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    with open(filename, 'rt') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        i = 0
        for row in csvreader:
            if len(row) == ATTRS_WO_CLASS + 1:
                i += 1
                instance = [row[i] for i in range(ATTRS_WO_CLASS)]  # first ATTRS_WO_CLASS values are attributes
                label = row[ATTRS_WO_CLASS]  # (ATTRS_WO_CLASS + 1)th value is the class label
                if i % 2 == 0:  # test instance
                    test_x.append(instance)
                    test_y.append(label)
                else:  # train instance
                    train_x.append(instance)
                    train_y.append(label)
                    
    return train_x, train_y, test_x, test_y

And then we can use it to load the data.

In [201]:
train_x, train_y, test_x, test_y = load_data("data/basketball.train.csv")

predictions evaluator:

In [202]:
def evaluate(predictions, true_labels):
    correct = 0
    incorrect = 0
    for i in range(len(predictions)):
        if predictions[i] == true_labels[i]:
            correct += 1
        else:
            incorrect += 1

    print("\tAccuracy:   ", correct / len(predictions))
    print("\tError rate: ", incorrect / len(predictions))
    
    return correct / len(predictions)

Scikit-learn needs that all the attribute values to be numeric. This is, we need to binarize all the non-numeric attribute values, to obtain vectors: records having only numbers. The `DictVectorizer` class provided by scikit-learn allows to do this easily.

In [203]:
from sklearn.feature_extraction import DictVectorizer

Mind that each `train_x` and `test_x` are a list of lists.

We just need to obtain from each a list of dictionaries (as done in previous practica where each record was a dictionary).

In [204]:
dicts_train_x = []
for x in train_x:
    d = {}
    for i, attr in enumerate(ATTRS):
        if i < len(ATTRS) - 1: # we removed class from train_x elems
            val = x[i]
            # save as floats the values for the already-numeric attributes from dataset, keep the rest as the strings they are
            if i not in [0, 1, 4, 10]:  # indices for "LOCATION", "W", "PERIOD", "PTS_TYPE" attributes
                val = float(val)
            d[attr] = val
    dicts_train_x.append(d)

Finally, the `fit_transform` method of the vectorizer binarizes the non-numeric attributes in the list of dictionaries, and returns the vector we need.

In [205]:
vectorizer_train = DictVectorizer()
vec_train_x = vectorizer_train.fit_transform(dicts_train_x).toarray()

We do similarly for vectorizing `test_x`.

In [206]:
dicts_test_x = []
# TODO DONE
for x in test_x:
    d = {}
    for i, attr in enumerate(ATTRS):
        if i < len(ATTRS) - 1: # we removed class from test_x elems
            val = x[i]
            # save as floats the values for the already-numeric attributes from dataset, keep the rest as the strings they are
            if i not in [0, 1, 4, 10]:  # indices for "LOCATION", "W", "PERIOD", "PTS_TYPE" attributes
                val = float(val)
            d[attr] = val
    dicts_test_x.append(d)

vectorizer_test = DictVectorizer()
vec_test_x = vectorizer_train.fit_transform(dicts_test_x).toarray()

Having `evaluate` defined somewhere, we are ready to learn and apply the model, similarly to Task 3 of Practicum 6. But here, we use the vectors recently obtained for the input sets. E.g., for Naive Bayes classifier:

In [207]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#clf = GaussianNB()
#clf.fit(vec_train_x, train_y)
#predictions = clf.predict(vec_test_x)
#evaluate(predictions, test_y)

In [208]:
classifiers = {"Decision Tree": DecisionTreeClassifier(),
               "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
               "Naive Bayes (Gaussian)": GaussianNB(),
               "Random Forests": RandomForestClassifier(n_estimators=50, max_features=5)  # number of trees in the forest, and maximum number of features in each tree
               }

best_accuracy = 0

for name, clf in classifiers.items():
    print(name)
    clf.fit(vec_train_x, train_y)
    predictions = clf.predict(vec_test_x)
    accuracy = evaluate(predictions, test_y)
    # print("\tPrediction list: ", predictions)
    # print("\tPrediction list length: %d" % len(predictions))
    if accuracy > best_accuracy: 
        best_accuracy = accuracy
        best_preds = predictions
        best_clf = name

print("\nBest classifier: %s" % best_clf)
        


Decision Tree
	Accuracy:    0.5363325525671055
	Error rate:  0.4636674474328945
Nearest Neighbors
	Accuracy:    0.5472363567832431
	Error rate:  0.4527636432167569
Naive Bayes (Gaussian)
	Accuracy:    0.57109011119188
	Error rate:  0.42890988880812
Random Forests
	Accuracy:    0.5780362383221603
	Error rate:  0.4219637616778397

Best classifier: Random Forests


In [209]:
with open(OUTPUT_FILE, 'w') as output:
    output.write("Id,Target")
    for id, pred in enumerate(best_preds):
        if (id + 1 <= 37142):
            output.write("\n%s,%s" % (id + 1, pred))
print("Results written to '%s'" % OUTPUT_FILE)

Results written to 'data/classifier_basketball.pred.csv'
